In [1]:
import os
os.chdir("/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project/cleaned_data")
print(os.getcwd())

/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project/cleaned_data


In [2]:
import numpy as np
import pandas as pd
import pandasql as ps

In [3]:
"""query = '''SELECT DISTINCT State, County FROM aqi'''
aqi_counties = ps.sqldf(query=query)
print('aqi',len(aqi_counties))

query = '''SELECT DISTINCT State, County FROM ghg'''
ghg_counties = ps.sqldf(query=query)
print('ghg',len(ghg_counties))

query = '''SELECT DISTINCT State, County FROM income'''
income_counties = ps.sqldf(query=query)
print('income',len(income_counties))

query = '''SELECT DISTINCT State, County FROM pop'''
pop_counties = ps.sqldf(query=query)
print('pop',len(pop_counties))

query = '''SELECT DISTINCT State, County FROM plant'''
plant_counties = ps.sqldf(query=query)
print('plant',len(plant_counties))

query = '''SELECT DISTINCT State, County FROM weather'''
weather_counties = ps.sqldf(query=query)
print('weather',len(weather_counties))"""

"query = '''SELECT DISTINCT State, County FROM aqi'''\naqi_counties = ps.sqldf(query=query)\nprint('aqi',len(aqi_counties))\n\nquery = '''SELECT DISTINCT State, County FROM ghg'''\nghg_counties = ps.sqldf(query=query)\nprint('ghg',len(ghg_counties))\n\nquery = '''SELECT DISTINCT State, County FROM income'''\nincome_counties = ps.sqldf(query=query)\nprint('income',len(income_counties))\n\nquery = '''SELECT DISTINCT State, County FROM pop'''\npop_counties = ps.sqldf(query=query)\nprint('pop',len(pop_counties))\n\nquery = '''SELECT DISTINCT State, County FROM plant'''\nplant_counties = ps.sqldf(query=query)\nprint('plant',len(plant_counties))\n\nquery = '''SELECT DISTINCT State, County FROM weather'''\nweather_counties = ps.sqldf(query=query)\nprint('weather',len(weather_counties))"

In [4]:
stop = True

In [5]:
os.chdir("/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project/cleaned_data")
aqi = pd.read_csv("aqi_2016.csv")
ghg = pd.read_csv("ghg_2016.csv")
income = pd.read_csv("income_2016.csv")
pop = pd.read_csv("population_2016.csv")
plant = pd.read_csv("powerplant_2016.csv")
weather = pd.read_csv("EPA_weather_2016.csv")

os.chdir("/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project/Github/Air-Quality-Prediction-Regression")
weather_meso = pd.read_csv("data/weather_data.csv",index_col=0)

NumCensusAreas = pd.read_csv("/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project/StatesNumCensusAreas.csv", header=0)
NumCensusAreas.head()

states = pd.read_csv("data/States.csv")

In [6]:
weather_meso['Date'] = pd.to_datetime(weather_meso['Date'])
weather_meso['Year'] = weather_meso['Date'].dt.year
weather_meso['Month'] = weather_meso['Date'].dt.month
weather_meso['Day'] = weather_meso['Date'].dt.day

weather_meso.head()

,County,Date,State,Unnamed: 0.1,air_temp_set_1,altimeter_set_1,precip_accum_24_hour_set_1,relative_humidity_set_1,wind_speed_set_1,Year,Month,Day
0,Sacramento,2016-01-01,CA,0.0,9.673846,102174.066000,0.0000,45.077273,1.459231,2016,1,1
1,Sacramento,2016-01-02,CA,1.0,8.212308,102086.020000,0.0000,49.951818,0.603846,2016,1,2
2,Sacramento,2016-01-03,CA,2.0,8.786364,101681.913333,0.0000,53.736667,0.822727,2016,1,3
3,Sacramento,2016-01-04,CA,3.0,12.060769,101131.062000,0.2032,48.968182,1.688462,2016,1,4
4,Sacramento,2016-01-05,CA,4.0,11.375833,100965.127500,2.7305,74.032000,2.935000,2016,1,5


In [7]:
weather_meso = pd.merge(weather_meso, states, how="left", left_on="State", right_on="Abbreviation")
weather_meso.drop({'State_x','Date','Unnamed: 0.1','Abbreviation'}, axis=1, inplace=True)
weather_meso.rename({'air_temp_set_1':'Temperature', 'altimeter_set_1':'Altimeter', 'precip_accum_24_hour_set_1':'Precipitation_24hrAccum','relative_humidity_set_1':'RelHumidity','wind_speed_set_1':'WindSpeed','State_y':'State'},axis=1,inplace=True)
weather_meso.head()

,County,Temperature,Altimeter,Precipitation_24hrAccum,RelHumidity,WindSpeed,Year,Month,Day,State
0,Sacramento,9.673846,102174.066000,0.0000,45.077273,1.459231,2016,1,1,California
1,Sacramento,8.212308,102086.020000,0.0000,49.951818,0.603846,2016,1,2,California
2,Sacramento,8.786364,101681.913333,0.0000,53.736667,0.822727,2016,1,3,California
3,Sacramento,12.060769,101131.062000,0.2032,48.968182,1.688462,2016,1,4,California
4,Sacramento,11.375833,100965.127500,2.7305,74.032000,2.935000,2016,1,5,California


In [8]:
#weather_meso = weather_meso.drop({'Date'}, axis=1)

In [9]:
weather.head()

,State,County,Date Local,State Code,County Code,Average Temperature,Year,Month,Day,Average Air Pressure,Average Humidity,Average Windspeed
0,alabama,jefferson,2016-01-01 00:00:00.000000,1,73,38.925000,2016,1,1,1002.581250,71.083333,145.589584
1,alabama,jefferson,2016-01-02 00:00:00.000000,1,73,38.339584,2016,1,2,1001.658334,67.083334,124.310417
2,alabama,jefferson,2016-01-03 00:00:00.000000,1,73,40.785416,2016,1,3,996.637500,67.020834,120.292708
3,alabama,jefferson,2016-01-04 00:00:00.000000,1,73,36.147917,2016,1,4,1000.566667,62.520833,84.986458
4,alabama,jefferson,2016-01-05 00:00:00.000000,1,73,37.666667,2016,1,5,1006.175000,41.687500,30.745833


In [10]:
aqi['State'] = aqi['State'].str.strip()
aqi['County'] = aqi['County'].str.strip()
ghg['State'] = ghg['State'].str.strip()
ghg['County'] = ghg['County'].str.strip()
income['State'] = income['State'].str.strip()
income['County'] = income['County'].str.strip()
pop['State'] = pop['State'].str.strip()
pop['County'] = pop['County'].str.strip()
plant['State'] = plant['State'].str.strip()
plant['County'] = plant['County'].str.strip()
weather['State'] = weather['State'].str.strip()
weather['County'] = weather['County'].str.strip()
weather_meso['State'] = weather_meso['State'].str.strip()
weather_meso['County'] = weather_meso['County'].str.strip()

weather_meso['State'] = weather_meso['State'].str.lower()
weather_meso['County'] = weather_meso['County'].str.lower()

aqi['County'] = aqi['County'].str.replace('-',' ', regex=True)
aqi['County'] = aqi['County'].str.replace('city and borough','', regex=True)
aqi['County'] = aqi['County'].str.replace(' borough','', regex=True)
aqi['County'] = aqi['County'].str.replace('\(city\)','city', regex=True)
aqi['County'] = aqi['County'].str.replace('municipality','', regex=True)
aqi['County'] = aqi['County'].str.replace('division','', regex=True)
aqi['County'] = aqi['County'].str.replace('census area','', regex=True)
aqi['County'] = aqi['County'].str.replace('parish','', regex=True)
aqi['County'] = aqi['County'].str.replace('st[.]','st ', regex=True)
aqi['County'] = aqi['County'].str.replace('saint  ','st ', regex=True)
aqi['County'] = aqi['County'].str.replace(' ','', regex=True)
aqi = aqi.dropna(subset=['State', 'County'])
aqi['County'] = aqi['County'].str.strip()


ghg['County'] = ghg['County'].str.replace('-',' ', regex=True)
ghg['County'] = ghg['County'].str.replace('city and borough','', regex=True)
ghg['County'] = ghg['County'].str.replace('\(city\)','city', regex=True)
ghg['County'] = ghg['County'].str.replace(' borough','', regex=True)
ghg['County'] = ghg['County'].str.replace('municipality','', regex=True)
ghg['County'] = ghg['County'].str.replace('division','', regex=True)
ghg['County'] = ghg['County'].str.replace('census area','', regex=True)
ghg['County'] = ghg['County'].str.replace('parish','', regex=True)
ghg['County'] = ghg['County'].str.replace('st[.]','st ', regex=True)
ghg['County'] = ghg['County'].str.replace('saint ','st ', regex=True)
ghg['County'] = ghg['County'].str.replace(' ','', regex=True)
ghg = ghg.dropna(subset=['State', 'County'])
ghg['County'] = ghg['County'].str.strip()


income['County'] = income['County'].str.replace('-',' ', regex=True)
income['County'] = income['County'].str.replace('city and borough','', regex=True)
income['County'] = income['County'].str.replace('\(city\)','city', regex=True)
income['County'] = income['County'].str.replace(' borough','', regex=True)
income['County'] = income['County'].str.replace('municipality','', regex=True)
income['County'] = income['County'].str.replace('division','', regex=True)
income['County'] = income['County'].str.replace('census area','', regex=True)
income['County'] = income['County'].str.replace('parish','', regex=True)
income['County'] = income['County'].str.replace('st[.]','st ', regex=True)
income['County'] = income['County'].str.replace('saint ','st ', regex=True)
income['County'] = income['County'].str.replace(' ','', regex=True)
income = income.dropna(subset=['State', 'County'])
income['County'] = income['County'].str.strip()


pop['County'] = pop['County'].str.replace('-',' ', regex=True)
pop['County'] = pop['County'].str.replace('city and borough','', regex=True)
pop['County'] = pop['County'].str.replace('\(city\)','city', regex=True)
pop['County'] = pop['County'].str.replace(' borough','', regex=True)
pop['County'] = pop['County'].str.replace('municipality','', regex=True)
pop['County'] = pop['County'].str.replace('division','', regex=True)
pop['County'] = pop['County'].str.replace('census area','', regex=True)
pop['County'] = pop['County'].str.replace('parish','', regex=True)
pop['County'] = pop['County'].str.replace('st[.]','st ', regex=True)
pop['County'] = pop['County'].str.replace('saint ','st ', regex=True)
pop['County'] = pop['County'].str.replace(' ','', regex=True)
pop = pop.dropna(subset=['State', 'County'])
pop['County'] = pop['County'].str.strip()


plant['County'] = plant['County'].str.replace('-',' ', regex=True)
plant['County'] = plant['County'].str.replace('city and borough','', regex=True)
plant['County'] = plant['County'].str.replace('\(city\)','city', regex=True)
plant['County'] = plant['County'].str.replace(' borough','', regex=True)
plant['County'] = plant['County'].str.replace('municipality','', regex=True)
plant['County'] = plant['County'].str.replace('division','', regex=True)
plant['County'] = plant['County'].str.replace('census area','', regex=True)
plant['County'] = plant['County'].str.replace('parish','', regex=True)
plant['County'] = plant['County'].str.replace('st[.]','st ', regex=True)
plant['County'] = plant['County'].str.replace('saint ','st ', regex=True)
plant['County'] = plant['County'].str.replace(' ','', regex=True)
plant = plant.dropna(subset=['State', 'County'])
plant['County'] = plant['County'].str.strip()


weather['County'] = weather['County'].str.replace('-',' ', regex=True)
weather['County'] = weather['County'].str.replace('city and borough','', regex=True)
weather['County'] = weather['County'].str.replace('\(city\)','city', regex=True)
weather['County'] = weather['County'].str.replace(' borough','', regex=True)
weather['County'] = weather['County'].str.replace('municipality','', regex=True)
weather['County'] = weather['County'].str.replace('division','', regex=True)
weather['County'] = weather['County'].str.replace('census area','', regex=True)
weather['County'] = weather['County'].str.replace('parish','', regex=True)
weather['County'] = weather['County'].str.replace('st[.]','st ', regex=True)
weather['County'] = weather['County'].str.replace('saint ','st ', regex=True)
weather['County'] = weather['County'].str.replace(' ','', regex=True)
weather = weather.dropna(subset=['State', 'County'])
weather['County'] = weather['County'].str.strip()

weather_meso['County'] = weather_meso['County'].str.replace('-',' ', regex=True)
weather_meso['County'] = weather_meso['County'].str.replace('city and borough','', regex=True)
weather_meso['County'] = weather_meso['County'].str.replace('\(city\)','city', regex=True)
weather_meso['County'] = weather_meso['County'].str.replace(' borough','', regex=True)
weather_meso['County'] = weather_meso['County'].str.replace('municipality','', regex=True)
weather_meso['County'] = weather_meso['County'].str.replace('division','', regex=True)
weather_meso['County'] = weather_meso['County'].str.replace('census area','', regex=True)
weather_meso['County'] = weather_meso['County'].str.replace('parish','', regex=True)
weather_meso['County'] = weather_meso['County'].str.replace('st[.]','st ', regex=True)
weather_meso['County'] = weather_meso['County'].str.replace('saint ','st ', regex=True)
weather_meso['County'] = weather_meso['County'].str.replace(' ','', regex=True)
weather_meso = weather_meso.dropna(subset=['State', 'County'])
weather_meso['County'] = weather_meso['County'].str.strip()


aqi_cty = aqi[['State','County']].drop_duplicates()
aqi_cty['aqi'] = 1
ghg_cty = ghg[['State','County']].drop_duplicates()
ghg_cty['ghg'] = 1
income_cty = income[['State','County']].drop_duplicates()
income_cty['income'] = 1
pop_cty = pop[['State','County']].drop_duplicates()
pop_cty['pop'] = 1
plant_cty = plant[['State','County']].drop_duplicates()
plant_cty['plant'] = 1
weather_cty = weather[['State','County']].drop_duplicates()
weather_cty['weather_epa'] = 1
weather_meso_cty = weather_meso[['State','County']].drop_duplicates()
weather_meso_cty['weather_meso'] = 1

In [11]:
check = pd.merge(aqi_cty, ghg_cty, on=['State', 'County'], how='outer')
check = pd.merge(check, income_cty, on=['State', 'County'], how='outer')
check = pd.merge(check, pop_cty, on=['State', 'County'], how='outer')
check = pd.merge(check, plant_cty, on=['State', 'County'], how='outer')
check = pd.merge(check, weather_cty, on=['State', 'County'], how='outer')
check = pd.merge(check, weather_meso_cty, on=['State', 'County'], how='outer')
check = check.sort_values(by=['State', 'County'], ascending=True)

display(check.head(10))

check.to_csv("data/data_merged_checkcounties.csv")

,State,County,aqi,ghg,income,pop,plant,weather_epa,weather_meso
1053,alabama,autauga,NaN,1.0,1.0,1.0,1.0,NaN,NaN
0,alabama,baldwin,1.0,1.0,1.0,1.0,NaN,NaN,NaN
1054,alabama,barbour,NaN,1.0,1.0,1.0,NaN,NaN,NaN
2206,alabama,bibb,NaN,NaN,1.0,1.0,NaN,NaN,NaN
2065,alabama,blount,NaN,1.0,1.0,1.0,NaN,NaN,NaN
2207,alabama,bullock,NaN,NaN,1.0,1.0,NaN,NaN,NaN
2208,alabama,butler,NaN,NaN,1.0,1.0,NaN,NaN,NaN
1055,alabama,calhoun,NaN,1.0,1.0,1.0,1.0,NaN,NaN
1056,alabama,chambers,NaN,1.0,1.0,1.0,1.0,NaN,NaN
2118,alabama,cherokee,NaN,1.0,1.0,1.0,1.0,NaN,NaN


In [25]:
data0 = aqi
data1 = pd.merge(data0, ghg, how="left", on = ["Year", "State", "County"])
data2 = pd.merge(data1, income, how="left", on = ["Year", "State", "County"])
data3 = pd.merge(data2, pop, how="left", on = ["Year", "State", "County"])
data4 = pd.merge(data3, plant, how="left", on = ["Year", "State", "County", "Month"])
data5 = pd.merge(data4, weather_meso, how="left", on=["Year", "State", "County", "Month", "Day"])
data7 = pd.merge(weather_meso, data4, how="left", on=["Year", "State", "County", "Month", "Day"])

data6 = data5.drop({'Date', 'Defining Site', 
                    'Number of Sites Reporting',
                   'StateNum', 'CountyNum', 'Division'}, axis=1)

data8 = data7.drop({'Date', 'Defining Site', 
                    'Number of Sites Reporting',
                   'StateNum', 'CountyNum', 'Division'}, axis=1)

data6.to_csv("data/data_merged_meso.csv")
data8.to_csv("data/data_merged_meso_weather-dropped.csv")
print(len(data6))
print(len(data8))

display(data6.head())
display(data8.head())

339949
368106


,State,County,State Code,County Code,AQI,Category,Defining Parameter,Year,Month,Day,...,Income,Region,Population,pp_consumed_MMBtu,pp_net_gen_MWh,Temperature,Altimeter,Precipitation_24hrAccum,RelHumidity,WindSpeed
0,alabama,baldwin,1,3,33,Good,PM2.5,2016,1,1,...,41286.0,3.0,207509.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alabama,baldwin,1,3,22,Good,PM2.5,2016,1,4,...,41286.0,3.0,207509.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,alabama,baldwin,1,3,28,Good,PM2.5,2016,1,7,...,41286.0,3.0,207509.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alabama,baldwin,1,3,13,Good,PM2.5,2016,1,10,...,41286.0,3.0,207509.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alabama,baldwin,1,3,36,Good,PM2.5,2016,1,13,...,41286.0,3.0,207509.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,County,Temperature,Altimeter,Precipitation_24hrAccum,RelHumidity,WindSpeed,Year,Month,Day,State,...,HFE,Short Lived Compounds,Other GHG,Biogenic CO2,Stationary Combustion,Income,Region,Population,pp_consumed_MMBtu,pp_net_gen_MWh
0,sacramento,9.673846,102174.066000,0.0000,45.077273,1.459231,2016,1,1,california,...,0.0,0.0,0.0,75181.5,75762.882,48122.0,4.0,1513260.0,5461752.0,717038.263
1,sacramento,8.212308,102086.020000,0.0000,49.951818,0.603846,2016,1,2,california,...,0.0,0.0,0.0,75181.5,75762.882,48122.0,4.0,1513260.0,5461752.0,717038.263
2,sacramento,8.786364,101681.913333,0.0000,53.736667,0.822727,2016,1,3,california,...,0.0,0.0,0.0,75181.5,75762.882,48122.0,4.0,1513260.0,5461752.0,717038.263
3,sacramento,12.060769,101131.062000,0.2032,48.968182,1.688462,2016,1,4,california,...,0.0,0.0,0.0,75181.5,75762.882,48122.0,4.0,1513260.0,5461752.0,717038.263
4,sacramento,11.375833,100965.127500,2.7305,74.032000,2.935000,2016,1,5,california,...,0.0,0.0,0.0,75181.5,75762.882,48122.0,4.0,1513260.0,5461752.0,717038.263


In [27]:
#Checking Number of Counties for each State
query = """SELECT State, COUNT(DISTINCT County) as NumCensusAreas from data6 GROUP BY State ORDER BY State ASC"""
check = ps.sqldf(query=query)

In [28]:
diff  = pd.merge(check, NumCensusAreas, on='State', how='left')
diff = diff.rename({'NumCensusAreas_x':'check', 'NumCensusAreas_y':'NumCensusAreas'}, axis=1)
diff['diff'] = diff['check'] - diff['NumCensusAreas']
diff[diff['diff'] > 0]

,State,check,NumCensusAreas,diff


In [13]:
data0 = aqi
print('aqi has',len(data0), 'lines of data')
print('aqi has',len(data0)-len(data0.dropna()), 'missing lines of data\n')

data1 = pd.merge(data0, ghg, how="left", on = ["Year", "State", "County"])
print('aqi+ghg has',len(data1), 'lines of data')
print('aqi+ghg has',len(data1)-len(data1.dropna()), 'missing lines of data\n')

data2 = pd.merge(data1, income, how="left", on = ["Year", "State", "County"])
print('aqi+ghg+income has',len(data2), 'lines of data')
print('aqi+ghg+income has',len(data2)-len(data2.dropna()), 'missing lines of data\n')

data3 = pd.merge(data2, pop, how="left", on = ["Year", "State", "County"])
print('aqi+ghg+income+pop has',len(data3), 'lines of data')
print('aqi+ghg+income+pop has',len(data3)-len(data3.dropna()), 'missing lines of data\n')

data4 = pd.merge(data3, plant, how="left", on = ["Year", "State", "County", "Month"])
print('aqi+ghg+income+pop+plant has',len(data4), 'lines of data')
print('aqi+ghg+income+pop+plant has',len(data4)-len(data4.dropna()), 'missing lines of data\n')

data5 = pd.merge(data4, weather, how="left", on=["Year", "State", "County", "Month", "Day"])
print('aqi+ghg+income+pop+plant+weather has',len(data5), 'lines of data')
print('aqi+ghg+income+pop+plant+weather has',len(data5)-len(data5.dropna()), 'missing lines of data\n')

data7 = pd.merge(weather, data4, how="left", on=["Year", "State", "County", "Month", "Day"])

data6 = data5.drop({'State Code_x', 'County Code_x', 'Date', 'Defining Site', 
                    'Number of Sites Reporting', 'Date Local', 'State Code_y', 'County Code_y',
                   'StateNum', 'CountyNum', 'Division'}, axis=1)

data8 = data7.drop({'State Code_x', 'County Code_x', 'Date', 'Defining Site', 
                    'Number of Sites Reporting', 'Date Local', 'State Code_y', 'County Code_y',
                   'StateNum', 'CountyNum', 'Division'}, axis=1)

aqi has 330533 lines of data
aqi has 0 missing lines of data

aqi+ghg has 339584 lines of data
aqi+ghg has 65654 missing lines of data

aqi+ghg+income has 339584 lines of data
aqi+ghg+income has 66414 missing lines of data

aqi+ghg+income+pop has 339584 lines of data
aqi+ghg+income+pop has 66414 missing lines of data

aqi+ghg+income+pop+plant has 339584 lines of data
aqi+ghg+income+pop+plant has 95214 missing lines of data

aqi+ghg+income+pop+plant+weather has 339584 lines of data
aqi+ghg+income+pop+plant+weather has 304168 missing lines of data



In [14]:
data6.to_csv("data/data_merged_epa.csv")
data8.to_csv("data/data_merged_epa_weather-dropped.csv")
print(len(data6))
print(len(data8))

339584
146532


In [15]:
query = ''' SELECT State, County, [Average Temperature], [Average Air Pressure], [Average Humidity], [Average Windspeed]
            FROM data6 
            WHERE [Average Temperature] IS NOT NULL
            AND [Average Air Pressure] IS NOT NULL
            AND [Average Humidity] IS NOT NULL
            AND [Average Windspeed] IS NOT NULL
            '''
check = ps.sqldf(query=query)
print('all aqi data =',len(data6))
print('all weather data present =',len(check))
print('missing =', len(data6) - len(check))
check.head()

all aqi data = 339584
all weather data present = 44572
missing = 295012


,State,County,Average Temperature,Average Air Pressure,Average Humidity,Average Windspeed
0,alabama,jefferson,38.925000,1002.581250,71.083333,145.589584
1,alabama,jefferson,38.339584,1001.658334,67.083334,124.310417
2,alabama,jefferson,40.785416,996.637500,67.020834,120.292708
3,alabama,jefferson,36.147917,1000.566667,62.520833,84.986458
4,alabama,jefferson,37.666667,1006.175000,41.687500,30.745833


In [16]:
query = '''SELECT DISTINCT State, County 
            FROM data6 
            WHERE [Average Temperature] IS NOT NULL
            AND [Average Air Pressure] IS NOT NULL
            AND [Average Humidity] IS NOT NULL
            AND [Average Windspeed] IS NOT NULL'''
check2 = ps.sqldf(query=query)
print(len(check2))

141


In [17]:
check2.head()

,State,County
0,alabama,jefferson
1,california,fresno
2,california,inyo
3,california,kern
4,california,losangeles


In [18]:
query = '''SELECT DISTINCT State, County,[Average Temperature],[Average Air Pressure],[Average Humidity],[Average Windspeed]
            FROM data6 
            WHERE [Average Temperature] IS NULL
            OR [Average Air Pressure] IS NULL
            OR [Average Humidity] IS NULL
            OR [Average Windspeed] IS NULL'''
missingweatherEPA = ps.sqldf(query=query)
print(len(missingweatherEPA))

91910


In [20]:
#Checking Number of Counties for each State
query = """SELECT State, COUNT(DISTINCT County) as NumCensusAreas from data6 GROUP BY State ORDER BY State ASC"""
check = ps.sqldf(query=query)

In [21]:
m = 5
n = 10

display(check[m:n])
display(NumCensusAreas[m:n])

,State,NumCensusAreas
5,colorado,32
6,connecticut,8
7,country of mexico,1
8,delaware,3
9,district of columbia,1


,State,NumCensusAreas
5,colorado,64
6,connecticut,8
7,delaware,3
8,district of columbia,1
9,florida,67


In [22]:
diff  = pd.merge(check, NumCensusAreas, on='State', how='left')
diff = diff.rename({'NumCensusAreas_x':'check', 'NumCensusAreas_y':'NumCensusAreas'}, axis=1)
diff['diff'] = diff['check'] - diff['NumCensusAreas']
diff[diff['diff'] > 0]

,State,check,NumCensusAreas,diff


In [23]:
assert stop != True

AssertionError: 

----------

In [ ]:
def missing_density(data, var):
    return (data[var].isnull()).sum()/data.shape[0]

In [ ]:
missing_density(data5, "Population")

In [ ]:
aqi.columns

In [ ]:
pop.columns

In [ ]:
aqi_pop = pd.merge(aqi, pop, on=['State', 'County', 'Year'], how='left')
#aqi_pop = pd.merge(aqi, pop, on=['State', 'County', 'Year'], how='outer')
print(len(aqi))
print(len(pop))
print(len(aqi_pop))

In [ ]:
query = '''SELECT State, County FROM aqi_pop WHERE Population IS NULL GROUP BY State, County'''
check = ps.sqldf(query=query)
check